In [6]:
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd
import time
import logging

In [7]:
#List of genres
genredata = requests.get("http://api.themoviedb.org/3/genre/movie/list?language=en-US&api_key=d34bc9fba7aa52b8156fd52d8f59bd08")
genredetails = genredata.json()
genre_df = json_normalize(genredetails['genres'])

In [8]:
#Fetch the dataset by genre
def getmoviesbygenre(movie_df, key, genre_id, year, count = 1000):
    data_size = requests.get("http://api.themoviedb.org/3/discover/movie?with_genres="
                             +genre_id+"&primary_release_year="+year+"&api_key="+key)
    temp_df = json_normalize(data_size.json())
    sleepcount = 0
    for i in range (1, temp_df.iloc[0][2]+1):
        if(len(movie_df) >= count):
            break
        if(sleepcount == 30):
            sleepcount = 0
            time.sleep(15)
        json_request = requests.get("http://api.themoviedb.org/3/discover/movie?with_genres="
                             +genre_id+"&primary_release_year="+year+"&page="+str(i)+"&api_key="+key)
        movie_df = movie_df.append(json_normalize(json_request.json()['results']))
        sleepcount = sleepcount + 1        
    return movie_df                                

In [9]:
#wrapper function for fetching the genres
def callgmbg():
    key ='d34bc9fba7aa52b8156fd52d8f59bd08'
    movie_df = pd.DataFrame()
    genre_id = [27, 18, 28, 9648, 878]
    for i in genre_id:
        movie_temp_df = pd.DataFrame()
        year = 2018
        while(len(movie_temp_df) < 1000):
            time.sleep(10)
            year = year - 1
            movie_temp_df = getmoviesbygenre(movie_temp_df, key, str(i), str(year), 1000)
            movie_df = movie_df.append(movie_temp_df)
    return movie_df

In [10]:
#Function to map genre names
def find_genre(genre_ids):
    in_genre = []
    for i in range(0, len(genre_ids)):
        if(genre_ids[i] in list(genre_df.id)):
            in_genre.append(genre_df.loc[genre_df['id'] == genre_ids[i], 'name'].item())
        else:
            continue
    return in_genre

In [11]:
#create genre dataset -- only this needs to be executed to get the final dataset
movie_df = callgmbg()
movie_df= movie_df.reset_index(drop=True)
#movie_df["tmdb_genre"] = "na"
movie_df["tmdb_genre"] = movie_df["genre_ids"].apply(find_genre)

In [69]:
counter = 0
movie_df['imdb_id'] = 'na'
#for i in range(0, len(movie_df)):
for i in range(0, 20):
    if(counter == 39):
        counter = 0
        time.sleep(15)
    data1 = requests.get("https://api.themoviedb.org/3/movie/" + str(movie_df.loc[i].id) +"?api_key=d34bc9fba7aa52b8156fd52d8f59bd08&language=en-US")
    if('imdb_id' in data1.json()):
        movie_df.set_value(i, 'imdb_id', json_normalize(data1.json())['imdb_id'][0][2:])
    else:
        continue

In [12]:
#creating csv
movie_df.to_csv("movie_df", sep=',', encoding='utf-8')

In [70]:
# Lets get the unique corresponding imdb ids from TMDB dataset and get the corresponding information we need from IMDB

movie_df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,tmdb_genre,imdb_id
0,False,/4G6FNNLSIVrwSRZyFs91hQ3lZtD.jpg,"[27, 53]",381288,en,Split,Though Kevin has evidenced 23 personalities to...,48.029324,/rXMWOZiCt6eMX22jWuTOSdQ98bY.jpg,2017-01-19,Split,False,6.8,1743,"[Horror, Thriller]",4972582
1,False,/hES8wGmkxHa54z7hqUMpw5TIs09.jpg,"[27, 878, 53]",395992,en,Life,The six-member crew of the International Space...,23.109515,/h2mhfbEBGABSHo2vXG1ECMKAJa7.jpg,2017-03-23,Life,False,6.1,240,"[Horror, Science Fiction, Thriller]",5442430
2,False,/aVyJnGubW0OBvqmqVg2z2wCuEsn.jpg,"[27, 9648, 53]",419430,en,Get Out,A young black man visits his white girlfriend'...,8.500050,/rdPGUJhadPg7FGFNzavib0iwTor.jpg,2017-02-24,Get Out,False,7.1,491,"[Horror, Mystery, Thriller]",5052448
3,False,/biN2sqExViEh8IYSJrXlNKjpjxx.jpg,[27],14564,en,Rings,"Julia becomes worried about her boyfriend, Hol...",6.890507,/bbxtz5V0vvnTDA2qWbiiRC77Ok9.jpg,2017-02-01,Rings,False,4.7,506,[Horror],0498381
4,False,/kMU8trT43p5LFoJ4plIySMOsZ1T.jpg,"[12, 28, 27, 878, 53]",126889,en,Alien: Covenant,Bound for a remote planet on the far side of t...,6.262471,/kwYzMCNUWUADZo9ox2jbUbcNT7O.jpg,2017-05-10,Alien: Covenant,False,0.0,4,"[Adventure, Action, Horror, Science Fiction, T...",2316204


## We hand picked here 5 imdb ids so that we can show how it works. 

In [71]:
movie_id_imdb = list(movie_df["imdb_id"].values)

In [59]:
#movie_id_20=[]
#for i in range(len(movie_id_imdb)) :
#    movie_id_20.append(movie_id_imdb[i][2:] )
    

In [72]:
movie_id =movie_id_20[:5]

In [73]:
#populate an empty dataframe to hold 1000 movies at least to find the genre pairs distributions using a heatmap
df_1000 = pd.DataFrame(columns =['imdb_id', 'Rating','Genre','RunTimes','Votes','Cast','Director','writer'] )

from imdb import IMDb   
import numpy as np

ia = IMDb()

                                 
for i in range(len(movie_id)):
       
    get_movie =ia.get_movie(movie_id[i])
                                 
    title =get_movie.get('title')
    rating=get_movie.get('rating')
    if rating ==None :
        rating=0
    
    runtimes1=get_movie.get('runtimes')
    
    if runtimes1 ==None :
        runtimes1=[]
    
    runtimes =''
    for j in range(len(runtimes1)):
       #concatenate all genres before loading into the dataframe
       runtimes = runtimes + ',' + (runtimes1[j] )
        
    votes=get_movie.get('votes')
    
    genre1=get_movie.get('genres')
    
    genre =''
    for j in range(len(genre1)):
       #concatenate all genres before loading into the dataframe
       genre = genre + ',' + (genre1[j] )
                                 
    cast_names= ''  
    cast_list = get_movie.get('cast')
    if cast_list ==None :
        cast_list=[]
    for  name in range(len(cast_list)):
        cast_names = cast_names + ',' + cast_list[name]['name']
        

    writer_names =''
    writer_list = get_movie.get('writer')
    if writer_list ==None :
        writer_list=[]
    for writer in  range(len(writer_list)):
        writer_names = writer_names + ',' + writer_list[writer]['name']
                                 
    
    dir_list = get_movie.get('director')
    dir_names =''
    if dir_list ==None :
        dir_list=[]
    for dr in range(len(dir_list)):
        dir_names = dir_names + ',' + dir_list[dr]['name']
    
                                 
    rowd= len(df_1000) + i
    df_1000.loc[rowd] = np.array([movie_id[i],rating,genre[1:],runtimes[1:],votes,
                                  cast_names[1:],dir_names[1:],writer_names[1:]])

df_1000
        

,imdb_id,Rating,Genre,RunTimes,Votes,Cast,Director,writer
0,4972582,7.4,"Horror,Thriller",117,108683,"James McAvoy,Anya Taylor-Joy,Betty Buckley,Hal...",M. Night Shyamalan,M. Night Shyamalan
2,5442430,7.0,"Horror,Sci-Fi,Thriller",104,30931,"Hiroyuki Sanada,Ryan Reynolds,Rebecca Ferguson...",Daniel Espinosa,"Rhett Reese,Paul Wernick"
4,5052448,8.2,"Horror,Mystery",104,39312,"Daniel Kaluuya,Allison Williams,Catherine Keen...",Jordan Peele,Jordan Peele
6,0498381,4.5,"Drama,Horror",102,9514,"Matilda Anna Ingrid Lutz,Alex Roe,Johnny Galec...",F. Javier Gutiérrez,"David Loucka,Jacob Estes,Akiva Goldsman,David ..."
8,2316204,0,"Sci-Fi,Thriller",,None,"Katherine Waterston,Michael Fassbender,James F...",Ridley Scott,"John Logan,Dante Harper,Jack Paglen,Michael Gr..."


In [76]:
two_source_df = pd.merge(movie_df, df_1000, how='inner', on ="imdb_id")
two_source_df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,...,vote_count,tmdb_genre,imdb_id,Rating,Genre,RunTimes,Votes,Cast,Director,writer
0,False,/4G6FNNLSIVrwSRZyFs91hQ3lZtD.jpg,"[27, 53]",381288,en,Split,Though Kevin has evidenced 23 personalities to...,48.029324,/rXMWOZiCt6eMX22jWuTOSdQ98bY.jpg,2017-01-19,...,1743,"[Horror, Thriller]",4972582,7.4,"Horror,Thriller",117,108683,"James McAvoy,Anya Taylor-Joy,Betty Buckley,Hal...",M. Night Shyamalan,M. Night Shyamalan
1,False,/hES8wGmkxHa54z7hqUMpw5TIs09.jpg,"[27, 878, 53]",395992,en,Life,The six-member crew of the International Space...,23.109515,/h2mhfbEBGABSHo2vXG1ECMKAJa7.jpg,2017-03-23,...,240,"[Horror, Science Fiction, Thriller]",5442430,7.0,"Horror,Sci-Fi,Thriller",104,30931,"Hiroyuki Sanada,Ryan Reynolds,Rebecca Ferguson...",Daniel Espinosa,"Rhett Reese,Paul Wernick"
2,False,/aVyJnGubW0OBvqmqVg2z2wCuEsn.jpg,"[27, 9648, 53]",419430,en,Get Out,A young black man visits his white girlfriend'...,8.500050,/rdPGUJhadPg7FGFNzavib0iwTor.jpg,2017-02-24,...,491,"[Horror, Mystery, Thriller]",5052448,8.2,"Horror,Mystery",104,39312,"Daniel Kaluuya,Allison Williams,Catherine Keen...",Jordan Peele,Jordan Peele
3,False,/biN2sqExViEh8IYSJrXlNKjpjxx.jpg,[27],14564,en,Rings,"Julia becomes worried about her boyfriend, Hol...",6.890507,/bbxtz5V0vvnTDA2qWbiiRC77Ok9.jpg,2017-02-01,...,506,[Horror],0498381,4.5,"Drama,Horror",102,9514,"Matilda Anna Ingrid Lutz,Alex Roe,Johnny Galec...",F. Javier Gutiérrez,"David Loucka,Jacob Estes,Akiva Goldsman,David ..."
4,False,/kMU8trT43p5LFoJ4plIySMOsZ1T.jpg,"[12, 28, 27, 878, 53]",126889,en,Alien: Covenant,Bound for a remote planet on the far side of t...,6.262471,/kwYzMCNUWUADZo9ox2jbUbcNT7O.jpg,2017-05-10,...,4,"[Adventure, Action, Horror, Science Fiction, T...",2316204,0,"Sci-Fi,Thriller",,None,"Katherine Waterston,Michael Fassbender,James F...",Ridley Scott,"John Logan,Dante Harper,Jack Paglen,Michael Gr..."


We have presented as limited features as possible as we don't want to hit the databases heavily right now

## Imbalanced nature of the data. Approach taken:
The process of obtaining the data has been a key point in any data science problem. TMDB and IMDB have been used as main sources to this project. Initially a random collection of 1000 movies has been obtained from one of the movie databases to have a glance at the data. The movie genres (response variable) among these randomly picked movies have highlighted a situation of skewed distribution of the genres. Most importantly some genres are most occurring like Action movies compared to Musical movies. This skewed distribution will create a bias towards the majority class while predicting. In other words this is a classic Imbalanced problem

 There are different ways of tackling this imbalanced problem. Common ways are either under sampling the majority group or over sampling the minority group. There are some other techniques like SMOTE  to overcome this imbalanced dataset by synthesizing the data for minority classes from the available minority group data .Even before attempting this over sampling or under sampling, we made sure we at least sample a significant proportion of every genre in the main sample that we obtain for the above  procedures to work. From here on, as a next step we decided to collect 10,000 movies and make sure that we sample at least a minimum of 5% of a specific genre to be there in the main sample. For those genres which are nearly less than 5%, a choice is made to ignore them as they can be seen as potential outliers

Now that the sample we collected has at least significant proportions of genres available, some of the minority groups are over sampled which means that these minority samples are repeatedly sampled into the overall sample until a good proportions of the minority classes are seen. we didn't want to under sample the majority classes in a fear of the losing out the importance that this group brings to the model .

We at this point are not aiming to see the sample entirely balanced; however the idea is to at least making a significant oversampling now and assign the class weights in the model that we decide to use in the project to better weigh all classes accordingly.


## Data description:
The task given is to predict the movie genres mainly using IMDb and TMDb movie data. IMDB movie data is an online database of information related to films, television programs and video games, including cast, production crew, fictional characters, biographies, plot summaries, trivia and reviews, operated by IMDb.com. The features chosen from IMDb as follows. We are confined to very few columns over here. The main objective is to use features like mpaa ratings to predict genres, for eg. horror movies mostly fall into one mpaa category. Or sometimes like knowing genre coterie of an actor/writer/director can help in genre classification.Awards nominations can help the genre prediction too. Presented below are few of the features so far we collected and in the limit of not hitting the api frequenclty, for this milestone we cameup with these features in the notebook

IMDB_ID: it is required to join the data between IMDb and TMDb database

Rating: The movie rating is required to answer the question what is the average rating of movies in different genres.
Genre:  To train the data for supervised learning.
RunTimes:  To check the correlation between genre and run time.  Usually dramatic movies will end up in longer run times
Votes: The votes are required to answer the question which genre is more popular than other.
Cast:  To answer question most actors are confined to  certain Genres. This helps in predicting the movie genre
Director:  To check the correlation between genre and Director. This helps in predicting the movie genre
Writer:  To check the correlation between genre and Writer.  This helps in predicting the movie genre
Parental_Guidance : To check the correlation between genre and Parental_Guidance.  

TDMB database is required as supplement to IMDB database. It mainly assists for answering our second task “to predict movie genres from posters”.
IMDB_ID : To join the data from TMDb
Revenue:  To answer our question about “Total revenue across all movies in a given genre year on year - Are some genres are losing the ability to attract viewers while others are gaining traction?
 Poster_Path: To download the poster and convert them into 1-D array and feed them as features for supervised Learning
Genres: To train the data for supervised learning and also to group them based on TDMB and IMDb genres.
Release_date: To answer revenue across movie in a given genre.

The poster url will then be converted into standardized pixels and be added to the merged dataset obtained from imdb and tmdb which completes the full dataset

### Please check the other pdf document where we have explained how we explained the preditors and how we finalized Y definitions and the sampling strategies. This includes some tables and some other images